In [10]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.optim as optim

import utils
import tasks_sine, tasks_celebA
from models import CaviaModel, Model_Active, Encoder_Decoder, Onehot_Encoder
from logger import Logger
import IPython

import random

def set_seed(seed, cudnn=True):
    """
    Seed everything we can!
    Note that gym environments might need additional seeding (env.seed(seed)),
    and num_workers needs to be set to 1.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # note: the below slows down the code but makes it reproducible
    if (seed is not None) and cudnn:
        torch.backends.cudnn.deterministic = True


In [87]:
!which python

/Users/huh/miniconda3/envs/bfs/bin/python


In [57]:

class manual_optim():
    def __init__(self, param_list, lr):
        self.param_list = param_list
        self.lr = lr
        
    def zero_grad(self):
        for par in self.param_list:
            par.grad = torch.zeros(par.data.shape)
        
    def step(self):
    
        for par in self.param_list:
    #         par.grad = torch.autograd.grad(task_loss, par, create_graph= not first_order)[0] #not args.first_order)[0]
            par.data = par.data - par.grad * self.lr

In [84]:

lr_inner = 0.01
first_order = True
tasks_per_metaupdate = 1
k_meta_train = 10
k_meta_test = 10

    
def eval_model(model, context, inputs, target_fnc):
    outputs = model(inputs, context)
    targets = target_fnc(inputs)
    return F.mse_loss(outputs, targets)

def inner_update(model, task_train, target_fnc):
    context = model.reset_context()
    optim_inner = optim.SGD([context], lr_inner) 
#     optim_inner = manual_optim([context], lr_inner) 
    train_inputs = task_train.sample_inputs(k_meta_train, False)
    for _ in range(1):
        optim_inner.zero_grad()
        task_loss = eval_model(model, context, train_inputs, target_fnc)
        context.grad = torch.autograd.grad(task_loss, context, create_graph= not first_order)[0] #not args.first_order)[0]
#         task_loss.backward(create_graph= True, retain_graph = False)
        optim_inner.step()
        
        print(context.grad)
        
    return context 


def get_meta_loss( model, task_family, target_fnc):
    meta_loss = 0
    for t in range(tasks_per_metaupdate):
        context = inner_update(model, task_family, target_fnc)
        test_inputs = task_family.sample_inputs(k_meta_test, False)
        meta_loss += eval_model(model, context, test_inputs, target_fnc)
    return meta_loss / tasks_per_metaupdate


In [85]:
set_seed(0)

MODEL = Model_Active
model = MODEL(n_arch=[1,4,4,1],  n_context=2)

meta_optimiser = optim.Adam(model.parameters(), 0.01)

task_train = tasks_sine.RegressionTasksSinusoidal()
target_functions = task_train.sample_tasks(tasks_per_metaupdate)[0]

meta_optimiser.zero_grad()
meta_loss = get_meta_loss( model, task_train, target_functions)
meta_loss.backward()

grad_norm = 0
for i, param in enumerate(model.parameters()):
    grad_norm += param.grad.norm()

print(grad_norm.item())


# meta_optimiser.step()

tensor([[-0.8015,  0.3827]])
4.098099708557129


In [ ]:
tensor([[-0.8015,  0.3827]])
tensor([[-0.7947,  0.3880]])
tensor([[-0.7878,  0.3930]])
tensor([[-0.7808,  0.3978]])
20.710269927978516

In [250]:

def get_meta_gradient(model, task_train, target_fnc):

    train_inputs = task_train.sample_inputs(k_meta_train, False)
    test_inputs = task_train.sample_inputs(k_meta_test, False)

    context = model.reset_context()
    for _ in range(4):
        context = update_context(model, context, train_inputs, target_fnc)

    loss_meta = eval_model(model, context, test_inputs, target_fnc)
    return loss_meta


def meta_backward( model, task_family, target_fnc):
    loss_meta = 0
    for t in range(tasks_per_metaupdate):
        loss_meta += get_meta_gradient( model, task_family, target_fnc)
    return loss_meta / tasks_per_metaupdate


In [239]:
set_seed(0)

MODEL = Model_Active
model = MODEL(n_arch=[1,4,4,1],  n_context=2)

meta_optimiser = optim.Adam(model.parameters(), 0.01)

task_train = tasks_sine.RegressionTasksSinusoidal()
target_functions = task_train.sample_tasks(tasks_per_metaupdate)[0]


meta_optimiser.zero_grad()

loss_meta = meta_backward( model, task_train, target_functions)
loss_meta.backward()
# 
grad_norm = 0
for i, param in enumerate(model.parameters()):
    grad_norm += param.grad.norm()

print(grad_norm.item())


4.03557014465332


In [ ]:
8.07114028930664
8.922293663024902
7.92586088180542

In [223]:
tasks_per_metaupdate

1